# Iceberg Workshop: Getting Started

## Setup Catalog

In [4]:
from pyiceberg.catalog import load_catalog

catalog = load_catalog("default")

## Example Data

| name    | id | date       |
|---------|----|------------|
| Alice   |  1 | 2018-04-02 |
| Bob     |  2 | 2020-09-07 |
| Charlie |  3 | 2022-07-01 |

In [9]:
import pyarrow as pa

example_schema = pa.schema(
    [
        ("name", pa.string()),
        ("id", pa.int32()),
        ("date", pa.date32()),
    ]
)

example_data = pa.Table.from_pylist([
    {"name": "Alice", "id": 1, "date": 17623},  # 2018-05-15
    {"name": "Bob", "id": 2, "date": 18512},    # 2020-11-23
    {"name": "Charlie", "id": 3, "date": 19174} # 2022-07-07
], schema=example_schema)

## Create an Iceberg table

First, we'll create a namespace `demo_ns` to organize the Iceberg table, then define and create the table using the specified schema.

In [5]:
catalog.create_namespace_if_not_exists("demo_ns")

In [14]:
table = catalog.create_table_if_not_exists("demo_ns.demo_table_1", schema=example_schema)
table

demo_table_1(
  1: name: optional string,
  2: id: optional int,
  3: date: optional date
),
partition by: [],
sort order: [],
snapshot: Operation.APPEND: id=2485838312579796827, parent_id=3672340927148582433, schema_id=0

## What happens behind table creation?

A metadata file has been created and registered as the latest metadata of table `demo_ns.demo_table_1`. Let's login to Minio Bucket and see the file:

- Minio Url: http://localhost:9001/
- username: admin
- password: password

The current bucket structure looks like the following

```
warehouse/
└── demo_ns/
    └── demo_table_1/
        └── metadata/
            └── <uuid>.metadata.json
```

# Add data to the table

It will create a new snapshot on the table

In [13]:
table.overwrite(example_data)
table

demo_table_1(
  1: name: optional string,
  2: id: optional int,
  3: date: optional date
),
partition by: [],
sort order: [],
snapshot: Operation.APPEND: id=2485838312579796827, parent_id=3672340927148582433, schema_id=0

## Read the table

We can see example data has been added to the table

In [11]:
table.scan().to_pandas()

,name,id,date
0,Alice,1,2018-04-02
1,Bob,2,2020-09-07
2,Charlie,3,2022-07-01


## What happens when adding data?

The data has been written into a parquet file and a new snapshot has been created.

```
warehouse/
└── demo_ns/
    └── demo_table_1/
        ├── metadata/
        │   ├── <uuid>.metadata.json
        │   ├── snap-<uuid>.avro
        │   └── <uuid>-m0.avro
        └── data/
            └── <uuid>.parquet

```

## Table Evolution: Make table partitioned

The table we just created is unpartitioned, but it's common practice to partition a table based on specific column(s). No worries—we can easily partition it!

Iceberg allows you to update the partitioning strategy without having to recreate the table or migrate any data.

In [ ]:
from pyiceberg.transforms import YearTransform

with table.update_spec() as update:
   update.add_field("date", YearTransform())

table

## Add more data to partitioned table

New data will be written in to different partitions based on the partition strategy. In this demo, there will be 3 partitions

- date_year=2018
- date_year=2020
- date_year=2022

TODO: Add a preview of current file structure

In [18]:
example_data_2 = pa.Table.from_pylist([
    {"name": "David", "id": 4, "date": 17623},  # 2018-05-15
    {"name": "John", "id": 5, "date": 18512},    # 2020-11-23
    {"name": "Jonas", "id": 6, "date": 19174} # 2022-07-07
], schema=example_schema)

table.append(example_data_2)

In [19]:
table.scan().to_pandas()

,name,id,date
0,David,4,2018-04-02
1,John,5,2020-09-07
2,Jonas,6,2022-07-01
3,Alice,1,2018-04-02
4,Bob,2,2020-09-07
5,Charlie,3,2022-07-01


# Table Evolution: Add new column to the table

Iceberg allows user to add new column(s) without re-create or re-write the table

In [21]:
from pyiceberg.types import StringType, DecimalType

with table.update_schema() as update:
   # Add new columns
   update.add_column("comments", StringType())
   update.add_column("salary", DecimalType(9, 3))

In [22]:
table

demo_table_1(
  1: name: optional string,
  2: id: optional int,
  3: date: optional date,
  4: comments: optional string,
  5: salary: optional decimal(9, 3)
),
partition by: [date_year],
sort order: [],
snapshot: Operation.APPEND: id=7312556961067351695, parent_id=2485838312579796827, schema_id=0

In [29]:
table.scan().to_pandas()

,name,id,date,comments,salary
0,David,4,2018-04-02,None,None
1,John,5,2020-09-07,None,None
2,Jonas,6,2022-07-01,None,None
3,Alice,1,2018-04-02,None,None
4,Bob,2,2020-09-07,None,None
5,Charlie,3,2022-07-01,None,None


## TODO: More Possible Topics
- Transaction
- Time travel
- Step-by-step exploration of table update

## Interoperability with other engines: Spark

Table created by pyiceberg can be consumed by Spark

In [68]:
from pyspark.sql import SparkSession

spark2 = (SparkSession.builder
  .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
  .config("spark.sql.catalog.spark_catalog", "org.apache.iceberg.spark.SparkSessionCatalog")
  .config("spark.jars.packages", "org.apache.iceberg:iceberg-spark-runtime-3.5_2.12:1.8.1,org.apache.iceberg:iceberg-aws-bundle:1.8.1")
  .config("spark.sql.catalog.demo.type", "rest")
  .config("spark.sql.catalog.demo", "org.apache.iceberg.spark.SparkCatalog")  
  .config("spark.sql.catalog.demo.uri", "http://rest:8181")
  .config("spark.sql.catalog.demo.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")       
  .config("spark.sql.catalog.demo.warehouse", "s3://warehouse")
  .config("spark.sql.catalog.demo.s3.endpoint", "http://localhost:9000")
  .config("spark.sql.catalog.demo.s3.region", "us-east-1")
  .config("spark.sql.catalog.demo.s3.access-key-id", "admin")
  .config("spark.sql.catalog.demo.s3.secret-access-key", "password")
  .config("spark.sql.catalog.demo.s3.path-style-access", "true")
).getOrCreate()

In [86]:
import os
import importlib.metadata
 # Remember to also update `dev/Dockerfile`
spark_version = ".".join(importlib.metadata.version("pyspark").split(".")[:2])
scala_version = "2.12"
iceberg_version = "1.5.0"

os.environ["PYSPARK_SUBMIT_ARGS"] = (
    f"--packages org.apache.iceberg:iceberg-spark-runtime-{spark_version}_{scala_version}:{iceberg_version},"
    f"org.apache.iceberg:iceberg-aws-bundle:{iceberg_version} pyspark-shell"
)
os.environ["AWS_REGION"] = "us-east-1"
os.environ["AWS_ACCESS_KEY_ID"] = "admin"
os.environ["AWS_SECRET_ACCESS_KEY"] = "password"

spark3 = (
        SparkSession.builder.appName("PyIceberg integration test")
        .config("spark.sql.session.timeZone", "UTC")
        .config("spark.sql.shuffle.partitions", "1")
        .config("spark.default.parallelism", "1")
        .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions")
        .config("spark.sql.catalog.integration", "org.apache.iceberg.spark.SparkCatalog")
        .config("spark.sql.catalog.integration.catalog-impl", "org.apache.iceberg.rest.RESTCatalog")
        .config("spark.sql.catalog.integration.cache-enabled", "false")
        .config("spark.sql.catalog.integration.uri", "http://rest:8181")
        .config("spark.sql.catalog.integration.io-impl", "org.apache.iceberg.aws.s3.S3FileIO")
        .config("spark.sql.catalog.integration.warehouse", "s3://warehouse/wh/")
        .config("spark.sql.catalog.integration.s3.endpoint", "http://minio:9000")
        .config("spark.sql.catalog.integration.s3.path-style-access", "true")
        .config("spark.sql.defaultCatalog", "integration")
        .config("spark.sql.execution.arrow.pyspark.enabled", "true")
        .getOrCreate()
    )

In [87]:
importlib.metadata.version("pyspark")

'3.5.0'

In [69]:
spark2.sql("USE demo")
spark2.sql("SHOW NAMESPACES").show()


+---------+
|namespace|
+---------+
|  demo_ns|
+---------+



In [70]:
spark2.sql("""CREATE TABLE IF NOT EXISTS demo.demo_ns.TEST_TABLE (
    id bigint NOT NULL COMMENT 'unique id',
    data string)
USING iceberg;
""")

DataFrame[]

In [88]:
spark3.sql("INSERT INTO integration.demo_ns.TEST_TABLE VALUES (1, 'some data'), (2, 'more data'), (3, 'yet more data')")


Py4JJavaError: An error occurred while calling o42.sql.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 2 in stage 17.0 failed 1 times, most recent failure: Lost task 2.0 in stage 17.0 (TID 53) (a03b84a7a4e2 executor driver): java.io.UncheckedIOException: Failed to close current writer
	at org.apache.iceberg.io.RollingFileWriter.closeCurrentWriter(RollingFileWriter.java:124)
	at org.apache.iceberg.io.RollingFileWriter.close(RollingFileWriter.java:147)
	at org.apache.iceberg.io.RollingDataWriter.close(RollingDataWriter.java:32)
	at org.apache.iceberg.spark.source.SparkWrite$UnpartitionedDataWriter.close(SparkWrite.java:747)
	at org.apache.iceberg.spark.source.SparkWrite$UnpartitionedDataWriter.commit(SparkWrite.java:729)
	at org.apache.spark.sql.execution.datasources.v2.WritingSparkTask.$anonfun$run$1(WriteToDataSourceV2Exec.scala:470)
	at org.apache.spark.util.Utils$.tryWithSafeFinallyAndFailureCallbacks(Utils.scala:1397)
	at org.apache.spark.sql.execution.dat

In [ ]:
spark.sql("SELECT * FROM demo.demo_ns.demo_table_1").show()

In [ ]:
from pyiceberg.catalog import load_catalog
from datetime import date, datetime, timezone
import uuid
import pyarrow as pa
from pyiceberg.io.pyarrow import pyarrow_to_schema
from pyiceberg.schema import Schema
from pyiceberg.types import NestedField, StringType, IntegerType, BooleanType, DateType, DecimalType, LongType
from pyiceberg.partitioning import PartitionSpec, PartitionField
from pyiceberg.transforms import IdentityTransform, YearTransform
import time

In [42]:
import os

os.environ.get("AWS_ACCESS_KEY_ID")

'admin'